In [47]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpkun_76rs/pubring.gpg' created
gpg: /tmp/tmpkun_76rs/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


KeyboardInterrupt: ignored

In [48]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
import os, cv2, sklearn
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras import backend as K
from keras.regularizers import l2
from keras.layers import LeakyReLU
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils import np_utils

In [142]:
root_dir = os.getcwd() + "/drive/project/data/road-train-2+valid/sample/"
image_dir = root_dir + "train_sample/"
mask_dir = root_dir + "mask_sample/"

imgs, mask_imgs = [], []


for im in os.listdir(image_dir):
  imPath = image_dir + im
  maskPath = mask_dir + im.split('_')[0] + "_mask.png"
  
  imgs.append(load_image(imPath))
  mask_imgs.append(load_gray_image(maskPath))
  
  print(imPath, maskPath)


imgs = np.asarray(imgs)
mask_imgs = np.asarray(mask_imgs)

/content/drive/project/data/road-train-2+valid/sample/train_sample/11168_sat.jpg /content/drive/project/data/road-train-2+valid/sample/mask_sample/11168_mask.png
/content/drive/project/data/road-train-2+valid/sample/train_sample/8048_sat.jpg /content/drive/project/data/road-train-2+valid/sample/mask_sample/8048_mask.png
/content/drive/project/data/road-train-2+valid/sample/train_sample/6822_sat.jpg /content/drive/project/data/road-train-2+valid/sample/mask_sample/6822_mask.png
/content/drive/project/data/road-train-2+valid/sample/train_sample/3594_sat.jpg /content/drive/project/data/road-train-2+valid/sample/mask_sample/3594_mask.png
/content/drive/project/data/road-train-2+valid/sample/train_sample/1945_sat.jpg /content/drive/project/data/road-train-2+valid/sample/mask_sample/1945_mask.png
/content/drive/project/data/road-train-2+valid/sample/train_sample/2129_sat.jpg /content/drive/project/data/road-train-2+valid/sample/mask_sample/2129_mask.png
/content/drive/project/data/road-train

In [143]:
print(imgs.shape)
print(mask_imgs.shape)

(12, 1024, 1024, 3)
(12, 1024, 1024)


# Create Array of Windows and Labels

In [0]:
X = imgs
Y = mask_imgs

In [0]:
def load_mask(img,patch_size,stride,padding):
  list_patches = []
  imgwidth = img.shape[0]
  imgheight = img.shape[1]
  for i in range(padding,imgheight+padding,stride):
    for j in range(padding,imgwidth+padding,stride):
      if img[j,i]>0.5:
        list_patches.append(1)
      else:
        list_patches.append(0)
  return list_patches

In [0]:
# foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

# def value_to_class(v):
#     df = np.sum(v)
#     if df > foreground_threshold:
#         return 1
#     else:
#         return 0

# Extract patches from input images
img_patches = [img_crop(imgs[i], 16, 16, 16, 0) for i in range(imgs.shape[0])]
# gt_patches = [img_crop_gt(Y[i], 16, 16, 16) for i in range(X.shape[0])]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
# gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

X = np.asarray([img_patches[i] for i in range(len(img_patches))])
# Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

Y = np.asarray([load_mask(mask_imgs[i],16,16,0) for i in range(mask_imgs.shape[0])])
Y = np.reshape(Y,-1)
# Y = [load_mask(Y[i], 16, 16, 0) for i in range(Y.shape[0])]



In [147]:
print(X.shape)
print(Y.shape)


(49152, 16, 16, 3)
(49152,)


In [148]:
print(sum(Y))

2362


In [0]:
Y_2d = np_utils.to_categorical(Y, num_classes=2)

In [150]:
print(np.count_nonzero(Y_2d[:,1]))
print(Y_2d.shape)
print(X.shape)

2362
(49152, 2)
(49152, 16, 16, 3)


# Pavllo Model

In [155]:
pool_size = (2, 2)
reg = 1e-6 # L2 regularization factor (used on weights, but not biases)
nb_classes = 2

model = Sequential()

model.add(Convolution2D(64, 5, 5, # 64 5x5 filters
                        border_mode='same',
                        input_shape=(16,16, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(128, 3, 3, # 128 3x3 filters
                        border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(256, 3, 3, # 256 3x3 filters
                        border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(256, 3, 3, # 256 3x3 filters
                        border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=pool_size, border_mode='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, W_regularizer=l2(reg))) # Fully connected layer (128 neurons)
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))

model.add(Dense(nb_classes, W_regularizer=l2(reg)))
model.add(Activation('softmax'))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(16, 16, 3..., padding="same")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same")`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="same")`
/us

In [156]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 16, 16, 64)        4864      
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 8, 8, 128)         73856     
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 8, 8, 128)         0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 4, 4, 128)         0         
__________

In [0]:
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [0]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])


In [0]:
model.fit(X,Y_2d, batch_size=128, epochs=10)

Epoch 1/10
49152/49152 [==============================] - 182s 4ms/step - loss: 0.7762 - acc: 0.9514
Epoch 2/10
 4352/49152 [=>............................] - ETA: 2:44 - loss: 0.7037 - acc: 0.9561

In [0]:
Y_pred = model.predict(X)

In [138]:
print(Y_pred)

[[ 614655.5  -347407.25]
 [ 552827.5  -311512.44]
 [ 598537.2  -337612.56]
 ...
 [ 374072.03 -211167.28]
 [ 375134.16 -211914.72]
 [ 365200.97 -206155.44]]


In [0]:
def pred_2_label(prediction):
  label = np.zeros(prediction.shape[0])
  for i in range(prediction.shape[0]):
    if prediction[i,0]>prediction[i,1]:
      label[i] = 1
    else:
        label[i] = 0
  return label


In [134]:
Y_label = pred_2_label(Y_pred)

acc = sum(abs(Y-Y_label))/Y.shape
print(acc)
print(sum(Y_label))

[0.98730469]
20480.0


In [128]:
root_dir = os.getcwd() + "/drive/project/data/road-train-2+valid/sample_valid/"
valid_image_dir = root_dir + "img/"
valid_mask_dir = root_dir + "mask/"

valid_imgs, valid_mask_imgs = [], []


for im in os.listdir(valid_image_dir):
  imPath = valid_image_dir + im
  maskPath = valid_mask_dir + im.split('_')[0] + "_mask.png"
  
  valid_imgs.append(load_image(imPath))
  valid_mask_imgs.append(load_gray_image(maskPath))
  
  print(imPath, maskPath)

valid_imgs = np.asarray(valid_imgs)
valid_mask_imgs = np.asarray(valid_mask_imgs)
# X_valid = valid_imgs
# Y_valid = valid_mask_imgs

img_patches = [img_crop(valid_imgs[i], 16, 16, 16, 0) for i in range(valid_imgs.shape[0])]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])

X_valid = np.asarray([img_patches[i] for i in range(len(img_patches))])

Y_valid = np.asarray([load_mask(valid_mask_imgs[i],16,16,0) for i in range(valid_mask_imgs.shape[0])])
Y_valid = np.reshape(Y_valid,-1)

/content/drive/project/data/road-train-2+valid/sample_valid/img/1849_sat.jpg /content/drive/project/data/road-train-2+valid/sample_valid/mask/1849_mask.png


In [129]:
print(X_valid.shape)

(4096, 16, 16, 3)


In [0]:
Y_valid_pred = model.predict(X_valid)

In [131]:
Y_valid_label = pred_2_label(Y_valid_pred)
acc = sum(abs(Y_valid-Y_valid_label))/Y_valid_label.shape
print(acc)
print(sum(Y_valid_label))

[0.95654297]
4096.0


# Helper Functions

In [0]:
def load_image(filename):
    img = cv2.imread(filename)
    resized_image = cv2.resize(img, (72,72)) 
    return img

def load_gray_image(infilename):
    img = cv2.imread(infilename)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resized_image = cv2.resize(gray, (72,72)) 
    return gray
  
def grey_scale_conversion(img):
    new_img = np.zeros((img.shape[0],img.shape[1]))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i,j,1] == 1.0:
                new_img[i,j] = 1
    return new_img

def pad_image(data, padding):
    """
    Extend the canvas of an image. Mirror boundary conditions are applied.
    """
    if len(data.shape) < 3:
        # Greyscale image (ground truth)
        data = np.lib.pad(data, ((padding, padding), (padding, padding)), 'reflect')
    else:
        # RGB image
        data = np.lib.pad(data, ((padding, padding), (padding, padding), (0,0)), 'reflect')
    return data
    
def img_crop_gt(im, w, h, stride):
    """ Crop an image into patches (this method is intended for ground truth images). """
    assert len(im.shape) == 2, 'Expected greyscale image.'
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    for i in range(0,imgheight,stride):
        for j in range(0,imgwidth,stride):
            im_patch = im[j:j+w, i:i+h]
            list_patches.append(im_patch)
    return list_patches
    
def img_crop(im, w, h, stride, padding):
    """ Crop an image into patches, taking into account mirror boundary conditions. """
    assert len(im.shape) == 3, 'Expected RGB image.'
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    im = np.lib.pad(im, ((padding, padding), (padding, padding), (0,0)), 'reflect')
    for i in range(padding,imgheight+padding,stride):
        for j in range(padding,imgwidth+padding,stride):
            im_patch = im[j-padding:j+w+padding, i-padding:i+h+padding, :]
            list_patches.append(im_patch)
    return list_patches
    
def create_patches(X, patch_size, stride, padding):
    img_patches = np.asarray([img_crop(X[i], patch_size, patch_size, stride, padding) for i in range(X.shape[0])])
    # Linearize list
    img_patches = img_patches.reshape(-1, img_patches.shape[2], img_patches.shape[3], img_patches.shape[4])
    return img_patches
    
def create_patches_gt(X, patch_size, stride):
    img_patches = np.asarray([img_crop_gt(X[i], patch_size, patch_size, stride) for i in range(X.shape[0])])
    # Linearize list
    img_patches = img_patches.reshape(-1, img_patches.shape[2], img_patches.shape[3])
    return img_patches
    
def group_patches(patches, num_images):
    return patches.reshape(num_images, -1)

def extract_img_features(filename, stride):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size, stride, padding)
    X = np.asarray([img_patches[i] for i in range(len(img_patches))])
    return X

def mask_to_submission_strings(model, image_filename):
    """ Reads a single image and outputs the strings that should go into the submission file. """
    img_number = int(re.search(r"\d+", image_filename).group(0))
    Xi = load_image(image_filename)
    Xi = Xi.reshape(1, Xi.shape[0], Xi.shape[1], Xi.shape[2])
    Zi = model.classify(Xi)
    Zi = Zi.reshape(-1)
    patch_size = 16
    nb = 0
    print("Processing " + image_filename)
    for j in range(0, Xi.shape[2], patch_size):
        for i in range(0, Xi.shape[1], patch_size):
            label = int(Zi[nb])
            nb += 1
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))

